# Aprendizaje Estadístico y Data Mining

## Práctica 2: Patrones Secuenciales

### Objetivo
El conjunto de datos **“Navegación_Web.csv”** contiene clientes y las acciones que hace en una Web, teniendo estas asociadas un timestamp.

* [Link al dataset.](./data/Navegacion_Web.csv)

**Enunciado:** Habrá que procesar el fichero para crear una estructura de datos de los distintos clientes con las secuencias de las acciones realizadas en distintos instantes 

**Solución**

Referencias:
- [Mostrar dataframes (GeeksForGeeks).](https://www.geeksforgeeks.org/how-to-pretty-print-an-entire-pandas-series-or-dataframe/)
- [Mostrar dataframes (Analytics Vidhya).](https://www.analyticsvidhya.com/blog/2021/06/style-your-pandas-dataframe-and-make-it-stunning/#h-styling-the-dataframe)
- [Mostrar dataframes (Make your tables look glorious).](https://medium.com/towards-data-science/make-your-tables-look-glorious-2a5ddbfcc0e5)
- [Imprimir diccionarios como JSON (Favtutor).](https://favtutor.com/blogs/pretty-print-dictionary-python)


En primer lugar, realiremos un Dataframe a partir del csv indicado. De esta manera, podremos trabajar más comodamente con los datos,

In [2]:
import copy
import json

import pandas as pd
from gsppy.gsp import GSP

In [134]:
CSV_FILE_PATH = "data/Navegacion_Web.csv"

def cargar_datos() -> pd.DataFrame:
    """Carga los datos del archivo CSV_FILE_PATH

    Returns:
        pd.DataFrame: DataFrame con los datos del archivo CSV_FILE_PATH
    """

    # 1. Leemos el archivo CSV y lo cargamos en un DataFrame
    df = pd.read_csv(CSV_FILE_PATH, sep=",")

    # 2. Como la columna 'Timestamp' está en formato string, la convertimos a datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # 3. Ordenamos el DataFrame por 'UserID' y 'Timestamp'
    df = df.sort_values(['UserID', 'Timestamp'])

    # 4. Devolvemos el DataFrame
    return df

def procesar_secuencias(df: pd.DataFrame) -> dict:
    """Procesa las secuencias de acciones de los usuarios

    Args:
        df (pd.DataFrame): DataFrame con los datos de las acciones de los usuarios

    Returns:
        dict: Lista de secuencias de acciones por
    """
    secuencias = {}

    # 1. Vamos a agrupar las acciones por 'UserID'
    for usuario, grupo in df.groupby('UserID'):

        # 1.1 Como el dataframe esta ordenado por 'UserID' y 'Timestamp',
        #     podemos obtener la secuencia de acciones simplemente cogiendo
        #     los valores de la columna 'Action' y convirtiéndolos a una lista
        secuencia = grupo['Action'].tolist()

        # 1.2 Añadimos la secuencia a la lista de secuencias
        secuencias.update({usuario: secuencia})

    # 2. Devolvemos el diccionario de secuencias
    return secuencias

# Cargamos los datos
df_datos = cargar_datos()

# Mostramos los datos
display(df_datos)

,UserID,Timestamp,Action
5,User_1,2024-01-01 12:46:08,Wishlist
1,User_1,2024-01-01 12:46:09,View
39,User_1,2024-01-01 12:46:10,Add to Cart
59,User_1,2024-01-01 12:46:16,Purchase
51,User_1,2024-01-01 12:46:18,Purchase
43,User_1,2024-01-01 12:46:19,Wishlist
3,User_1,2024-01-01 12:46:38,Purchase
45,User_10,2024-01-01 13:26:00,Contact
32,User_10,2024-01-01 13:26:02,Purchase
18,User_10,2024-01-01 13:26:12,Purchase


Ya con el Dataframe generado, procederemos con la creación de la estructura de datos. Para ello, tendremos en cuenta las transacciones realizadas por cada usuario en cada instante de tiempo, de tal manera que podamos averiguar la secuencia de acciones del mismo. La estructura que estaremos utilizando será un diccionario, cuya *key* sea el ID del usuario, y su valor una lista (que representará la secuencia) de listas (que representarán las transacciones).

In [135]:
secuencias_usuarios = procesar_secuencias(df_datos)
secuencias = [actions for actions in secuencias_usuarios.values()]

print(f"Secuencia de acciones de los usuarios: {json.dumps(secuencias_usuarios, indent=4)}")
print("\n")
print(f"Secuencias de acciones: {json.dumps(secuencias, indent=4)}")

Secuencia de acciones de los usuarios: {
    "User_1": [
        "Wishlist",
        "View",
        "Add to Cart",
        "Purchase",
        "Purchase",
        "Wishlist",
        "Purchase"
    ],
    "User_10": [
        "Contact",
        "Purchase",
        "Purchase",
        "Add to Cart",
        "Contact"
    ],
    "User_2": [
        "Contact",
        "View",
        "View",
        "Purchase",
        "View"
    ],
    "User_3": [
        "View",
        "Contact",
        "Contact",
        "Add to Cart",
        "View",
        "Logout"
    ],
    "User_4": [
        "Search",
        "Add to Cart",
        "Contact",
        "Add to Cart",
        "View",
        "Contact"
    ],
    "User_5": [
        "Add to Cart",
        "Purchase",
        "View",
        "Add to Cart",
        "Wishlist",
        "Contact"
    ],
    "User_6": [
        "Wishlist",
        "View",
        "Add to Cart",
        "Purchase",
        "View",
        "Search",
        "View"
    ]

Una vez hecho esto y analizando esta información se pueden extraer conclusiones sobre en qué orden compran los clientes los productos y así tomar decisiones de negocio. Este estudio se puede llevar a cabo aplicando el algoritmo Generalized Sequential Patterns utilizando la implementación de éste disponible en la librería *gsppy*.

**Enunciado:** Prueba al menos tres configuraciones de soporte diferentes. ¿Qué diferencias hay en los resultados y a que se debe?

**Solución**

En primer lugar, debemos transformar nuestro diccionario a un formato válido para gsppy. Para ello, realizaremos una lista apartir del diccionario.

In [136]:
def mostrar_patrones(resultado: dict, soporte_minimo: float, inicio: int = 0) -> None:
    """Muestra los patrones encontrados por el algoritmo GSP

    Args:
        resultado (dict): Diccionario con los patrones encontrados por el algoritmo GSP
        soporte_minimo (int): Soporte mínimo de los patrones encontrados
        inicio (int, optional): Indice de inicio para enumerar los patrones. Por defecto es 0.
    """
    print(f"Soporte mínimo {soporte_minimo}:")

    for tamano, patron in enumerate(resultado[inicio:], start=inicio):
        print(f"  Patron de tamaño {tamano + 1}:")
        for patron, frecuencia in patron.items():
            patron_str = ' -> '.join(patron)
            print(f"   * {patron_str}: {frecuencia} ocurrencias")
        print("\n")

resultado_sop_02 = GSP(secuencias).search(min_support=0.2)
mostrar_patrones(resultado_sop_02, 0.2)

Soporte mínimo 0.2:
  Patron de tamaño 1:
   * Wishlist: 5 ocurrencias
   * View: 7 ocurrencias
   * Add to Cart: 9 ocurrencias
   * Purchase: 8 ocurrencias
   * Contact: 7 ocurrencias
   * Logout: 2 ocurrencias
   * Search: 2 ocurrencias


  Patron de tamaño 2:
   * Wishlist -> View: 2 ocurrencias
   * View -> Add to Cart: 4 ocurrencias
   * View -> Contact: 2 ocurrencias
   * Add to Cart -> View: 2 ocurrencias
   * Add to Cart -> Purchase: 6 ocurrencias
   * Add to Cart -> Contact: 3 ocurrencias
   * Purchase -> Wishlist: 3 ocurrencias
   * Purchase -> View: 3 ocurrencias
   * Purchase -> Add to Cart: 2 ocurrencias
   * Contact -> Add to Cart: 4 ocurrencias


  Patron de tamaño 3:
   * Wishlist -> View -> Add to Cart: 2 ocurrencias
   * View -> Add to Cart -> Purchase: 2 ocurrencias
   * Add to Cart -> Purchase -> View: 2 ocurrencias
   * Add to Cart -> Contact -> Add to Cart: 2 ocurrencias
   * Contact -> Add to Cart -> View: 2 ocurrencias
   * Contact -> Add to Cart -> Purchase: 2 

Repetiremos este proceso, pero esta vez estableciendo el soporte a 0.3, de tal manera que sea un poco más estricto en cuanto a los valores que pasan y nos queden únicamente datos más relevantes en cuanto a frecuencia.

In [137]:
resultado_sop_03 = GSP(secuencias).search(min_support=0.3)
mostrar_patrones(resultado_sop_03, 0.3)

Soporte mínimo 0.3:
  Patron de tamaño 1:
   * Wishlist: 5 ocurrencias
   * View: 7 ocurrencias
   * Add to Cart: 9 ocurrencias
   * Purchase: 8 ocurrencias
   * Contact: 7 ocurrencias


  Patron de tamaño 2:
   * View -> Add to Cart: 4 ocurrencias
   * Add to Cart -> Purchase: 6 ocurrencias
   * Add to Cart -> Contact: 3 ocurrencias
   * Purchase -> Wishlist: 3 ocurrencias
   * Purchase -> View: 3 ocurrencias
   * Contact -> Add to Cart: 4 ocurrencias




Por último, ampliaremos un poco más el soporte a 0.4, para ver si limpamos más aún frecuencias irrelevantes.

In [138]:
resultado_sop_04 = GSP(secuencias).search(min_support=0.4)
mostrar_patrones(resultado_sop_04, 0.4)

Soporte mínimo 0.4:
  Patron de tamaño 1:
   * Wishlist: 5 ocurrencias
   * View: 7 ocurrencias
   * Add to Cart: 9 ocurrencias
   * Purchase: 8 ocurrencias
   * Contact: 7 ocurrencias


  Patron de tamaño 2:
   * View -> Add to Cart: 4 ocurrencias
   * Add to Cart -> Purchase: 6 ocurrencias
   * Contact -> Add to Cart: 4 ocurrencias




Al analizar los resultados que hemos obtenido con los diferentes valores de soporte mínimo, podemos obervar que la cantidad de patrones detectados disminuye a medida que incrementamos el umbral. Cuando tenemos un soporte mínimo de $0.2$, identificamos patrones de hasta tamaño $4$, mientras que con un soporte de $0.3$, solo encontramos patrones de hasta tamaño $2$, y con un soporte de $0.4$, reducimos aún más el resultado, eliminando aquellos de tamaño $3$ y $4$. Esto nos indica que muchas secuencias de eventos no son lo suficientemente frecuentes como para ser consideradas relevantes cuando exigimos un umbral de soporte más alto.

Otro aspecto a destacar es que ciertos patrones desaparecen a medida que el osoporte aumenta. Por ejemplo, en el soporte de $0.2$, existen secuencias como `"Wishlist → View → Add to Cart → Purchase"`, que desaparecen al aumentar el umbral a $0.3$ y $0.4$. Esto se debe a que estos patrones tienen una frecuencia menor y no alcanzan el número mínimo de ocurrencias que requieren los soportes más altos. En cambio, los patrones más frecuentes, como `"View → Add to Cart"`, `"Add to Cart → Purchase"` y `"Contact → Add to Cart"`, persisten en todas las configuraciones, lo que nos indica que representan comportamientos comunes de los usuarios.

Además, podemos notar que ciertas relaciones entre eventos, aunque son frecuentes en soportes bajos, dejan de ser consideradas a medida que elevamos el umbral. Por ejemplo, `"Add to Cart → Contact"`, que aparece en los resultados con soporte $0.2$ y $0.3$, ya no está presente en $0.4$, lo que nos sugiere que, aunque es relativamente común, su frecuencia no es lo suficientemente alta en comparación con otros patrones más recurrentes. Este comportamiento es esperable, ya que cuando exigimos un soporte mayor, solo se conservan los patrones más representativos y con mayor presencia en los datos.

**Enunciado:** Para una de ellas, interpreta algunos de los patrones secuenciales que te resulten curiosos.

**Solución**

Interpretaremos los patrones del primero de los estudios, el de soporte = 0.2, ya que es el único que tiene patrones de tamaño superior a 2, lo que hace que podamos encontrar secuencias más interesantes.  

En primer lugar, repetiremos el código, pero esta vez sin mostrar los patrones de tamaño 1, ya que no nos interesa.

In [139]:
mostrar_patrones(resultado_sop_02, 0.2, inicio=1)

Soporte mínimo 0.2:
  Patron de tamaño 2:
   * Wishlist -> View: 2 ocurrencias
   * View -> Add to Cart: 4 ocurrencias
   * View -> Contact: 2 ocurrencias
   * Add to Cart -> View: 2 ocurrencias
   * Add to Cart -> Purchase: 6 ocurrencias
   * Add to Cart -> Contact: 3 ocurrencias
   * Purchase -> Wishlist: 3 ocurrencias
   * Purchase -> View: 3 ocurrencias
   * Purchase -> Add to Cart: 2 ocurrencias
   * Contact -> Add to Cart: 4 ocurrencias


  Patron de tamaño 3:
   * Wishlist -> View -> Add to Cart: 2 ocurrencias
   * View -> Add to Cart -> Purchase: 2 ocurrencias
   * Add to Cart -> Purchase -> View: 2 ocurrencias
   * Add to Cart -> Contact -> Add to Cart: 2 ocurrencias
   * Contact -> Add to Cart -> View: 2 ocurrencias
   * Contact -> Add to Cart -> Purchase: 2 ocurrencias


  Patron de tamaño 4:
   * Wishlist -> View -> Add to Cart -> Purchase: 2 ocurrencias




Uno de los patrones que más nos ha llamado la atención es `"View → Add to Cart → Purchase"` (4 ocurrencias). Este patrón nos indica que los usuarios primero visualizan un producto, luego lo añaden al carrito y finalmente lo compran. Este es un comportamiento típico en un proceso de compra online, donde los clientes suelen ver y examinar los productos antes de decidir comprarlos. Este patrón nos sugiere que los usuarios que realizan estas tres acciones están interesados en el producto y están dispuestos a completar la transacción.

Otro patrón que nos ha parecido interesante es `Purchase → Wishlist` (3 ocurrencias). Lo habitual sería que los usuarios añadieran productos a la wishlist antes de comprarlos, pero este patrón nos sugiere que algunos clientes, tras realizar una compra, guardan otros productos en su wishlist. Esto puede indicar que, después de completar una transacción, algunos usuarios ya se están planeando futuras compras o que encuentran productos que les intereasn mientras navegaban por la tienda.

En el caso de patrones más largos, encontramos `"Wishlist → View → Add to Cart → Purchase"` (2 ocurrencias). Este patrón sigue un flujo de compra similar al primero, pero con un paso adicional: antes de añadir el producto al carrito, los usuarios lo guardan en la wishlist. Este patrón nos sugieren que algunos clientes utilizan la wishlist como una forma de guardar productos que les interesan y luego los revisan y compran.

**Enunciado:** ¿Qué transacción es clave eliminar para cambiar los patrones obtenidos en el punto anterior? ¿Y cuál de las secuencias?

**Solución**

Vamos a iterar sobre todas las transacciones y secuencias para ver  cuál afecta más a los patrones obtenidos. Para ello, eliminaremos una transacción a la vez y veremos cómo cambian los resultados. Luego, haremos lo mismo con las secuencias, eliminando una a una y observando los cambios.

In [ ]:
def comparar_patrones(patrones1: dict, patrones2: dict) -> tuple:
    """Compara dos conjuntos de patrones y devuelve las diferencias.

    Args:
        patrones1 (dict): Patrones GSP del primer conjunto
        patrones2 (dict): Patrones GSP del segundo conjunto

    Returns:
        tuple: Diferencia total, patrones que solo están en el primer conjunto, patrones que solo están en el segundo conjunto
    """

    # Convertimos los patrones a conjuntos de tuplas para poder compararlos
    set1 = set(map(tuple, [sorted(p) for p in patrones1]))
    set2 = set(map(tuple, [sorted(p) for p in patrones2]))

    # Vemos los patrones que solo están en uno de los conjuntos
    solo_en_1 = set1 - set2
    solo_en_2 = set2 - set1

    # Calculamos el total de diferencias
    diferencia = len(solo_en_1) + len(solo_en_2)

    return diferencia, solo_en_1, solo_en_2

# Ejecutamos GSP con soporte mínimo de 0.2 para sacar los patrones de referencia
patrones_referencia = GSP(secuencias).search(min_support=0.2)

# Lista para guardar los resultados
resultados = []

# Probar eliminando cada secuencia una por una
for secuencia in range(len(secuencias)):
    secuencias_prueba = copy.deepcopy(secuencias)
    secuencia_eliminada = secuencias_prueba.pop(secuencia)

    patrones_prueba = GSP(secuencias_prueba).search(min_support=0.2)
    diferencia, solo_original, solo_nuevo = comparar_patrones(patrones_referencia, patrones_prueba)

    resultados.append({
        'indice': secuencia,
        'secuencia': secuencia_eliminada,
        'diferencia': diferencia,
        'solo_original': solo_original,
        'solo_nuevo': solo_nuevo
    })

# Ordenamos por mayor impacto (mayor diferencia primero)
resultados.sort(key=lambda x: x['diferencia'], reverse=True)

# Mostramos las dos secuencias más influyentes
print("Las dos secuencias más influyentes son:")
for i in range(min(2, len(resultados))):
    print(f"\nSecuencia #{resultados[i]['indice']}: {resultados[i]['secuencia']}")
    print(f"Al eliminarla, hay {resultados[i]['diferencia']} cambios en los patrones")
    print("Patrones que desaparecen:")
    for patron in resultados[i]['solo_original']:
        print(f"  {patron}")
    print("Nuevos patrones que aparecen:")
    for patron in resultados[i]['solo_nuevo']:
        print(f"  {patron}")

# Probamos eliminando las dos secuencias más influyentes juntas
if len(resultados) >= 2:
    secuencias_prueba = copy.deepcopy(secuencias)
    indices_a_eliminar = [resultados[0]['indice'], resultados[1]['indice']]

    # Eliminamos de mayor a menor índice para evitar problemas
    for idx in sorted(indices_a_eliminar, reverse=True):
        secuencia_eliminada = secuencias_prueba.pop(idx)

    patrones_prueba = GSP(secuencias_prueba).search(min_support=0.2)
    diferencia, solo_original, solo_nuevo = comparar_patrones(patrones_referencia, patrones_prueba)

    print("\n\nAl eliminar las dos secuencias más influyentes juntas:")
    print(f"Hay {diferencia} cambios en los patrones")
    print("Patrones que desaparecen:")
    for patron in solo_original:
        print(f"  {patron}")
    print("Nuevos patrones que aparecen:")
    for patron in solo_nuevo:
        print(f"  {patron}")

print("\n\nPatrones originales:")
mostrar_patrones(patrones_referencia, 0.2, inicio=1)

# Ejecutamos GSP sin las dos secuencias más influyentes
if len(resultados) >= 2:
    secuencias_prueba = copy.deepcopy(secuencias)
    indices_a_eliminar = [resultados[0]['indice'], resultados[1]['indice']]

    # Eliminamos de mayor a menor índice
    for idx in sorted(indices_a_eliminar, reverse=True):
        secuencias_prueba.pop(idx)

    patrones_prueba = GSP(secuencias_prueba).search(min_support=0.2)
    print("\n\nPatrones después de eliminar las dos secuencias más influyentes:")
    mostrar_patrones(patrones_prueba, 0.2, inicio=1)

Las dos secuencias más influyentes son:

Secuencia #6: ['Wishlist', 'View', 'Add to Cart', 'Purchase', 'View', 'Search', 'View']
Al eliminarla, hay 7 cambios en los patrones
Patrones que desaparecen:
  (('Add to Cart', 'Contact', 'Add to Cart'), ('Add to Cart', 'Purchase', 'View'), ('Contact', 'Add to Cart', 'Purchase'), ('Contact', 'Add to Cart', 'View'), ('View', 'Add to Cart', 'Purchase'), ('Wishlist', 'View', 'Add to Cart'))
  (('Add to Cart',), ('Contact',), ('Logout',), ('Purchase',), ('Search',), ('View',), ('Wishlist',))
  (('Add to Cart', 'Contact'), ('Add to Cart', 'Purchase'), ('Add to Cart', 'View'), ('Contact', 'Add to Cart'), ('Purchase', 'Add to Cart'), ('Purchase', 'View'), ('Purchase', 'Wishlist'), ('View', 'Add to Cart'), ('View', 'Contact'), ('Wishlist', 'View'))
  (('Wishlist', 'View', 'Add to Cart', 'Purchase'),)
Nuevos patrones que aparecen:
  (('Add to Cart', 'Contact'), ('Add to Cart', 'Purchase'), ('Add to Cart', 'View'), ('Contact', 'Add to Cart'), ('Purchase'

Basándonos en los resultados, las dos secuencias más influyentes son:
* Secuencia $\#6$: `['Wishlist', 'View', 'Add to Cart', 'Purchase', 'View', 'Search', 'View']`
* Secuencia $\#3$: `['View', 'Contact', 'Contact', 'Add to Cart', 'View', 'Logout']`

Al eliminar cualquiera de estas dos secuencias individualmente cacambiamos cambia significativamente los patrones que hemos obtenido. Sin embargo, la secuencia $\#6$ tiene un impacto ligeramente mayor, desaparecen 7 patrones en comparación con los 6 patrones al eliminar la secuencia $\#3$. Esto nos indica que la secuencia $\#6$ tiene una mayor influencia en la estructura de los patrones secuenciales.

Cuando eliminamos ambas secuencias juntas, desaparecen 7 patrones y los cambios afectan tanto a patrones de tamaño 2 como de tamaño 3 y 4.

Al eliminar la secuencia $\#6$, desaparecen los siguientes patrones:
  * Patrones de tamaño 3 y 4 desaparecen, como:
      * `Wishlist → View → Add to Cart`
      * `View → Add to Cart → Purchase`
      * `Wishlist → View → Add to Cart → Purchase`
      * `Contact → Add to Cart → Purchase`
  * Disminuye la frecuencia de algunos patrones clave, como:
      * `Add to Cart → Purchase`
      * `View → Add to Cart`
      * `Contact → Add to Cart`

Esto significa que los usuarios que siguen el flujo de `wishlist → view → add to cart → purchase` contribuyen fuertemente a la formación de patrones de compra. Al eliminar esta secuencia, reducimos las apariciones de estos eventos secuenciales, haciendo que los patrones desaparezcan.

Por otro lado, la secuencia $\#3$ `(['View', 'Contact', 'Contact', 'Add to Cart', 'View', 'Logout'])` afecta patrones relacionados con la interacción con Contact. Eliminarla afecta a secuencias como:
* `View → Contact`
* `Contact → Add to Cart`
* `Add to Cart → Purchase`
* `Wishlist → View → Add to Cart`

Esto nos muestra que los usuarios que interactúan con "Contact" antes de agregar al carrito y comprar tienen un peso importante en la formación de patrones.

Las transacciones que debemos eliminar para cambiar los patrones obtenidos son:
* Basándonos en la secuencia $\#6$: -> User_6
  * `2024-01-01 12:29:57, Wishlist`
  * `2024-01-01 12:30:17, View`
  * `2024-01-01 12:30:18, Add to Cart`
  * `2024-01-01 12:30:20, Purchase`
  * `2024-01-01 12:30:45, Search`
  * `2024-01-01 12:30:46, View`

* Basándonos en la secuencia $\#3$ -> User_3
  * `2024-01-01 12:07:00, View`
  * `2024-01-01 12:07:23, Contact`
  * `2024-01-01 12:07:24, Contact`
  * `2024-01-01 12:07:26, Add to Cart`
  * `2024-01-01 12:07:56, View`
  * `2024-01-01 12:07:57, Logout`